<span style="color:black"><span style = "font-size:30px"> Critical subregion identification</span>
&nbsp;&nbsp;&nbsp;
   
    
In this process, we identified the critical subregion for promoter strength by varying promoter sequences and assessing its impacts. The CNN model (CNN training.ipynb) from the previous step is necessary to run this program. 

1. Import python modules and the constructed CNN model and load the native promoter dataset.

In [ ]:
# 1.

from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy.stats import pearsonr
from tqdm import tqdm
data = pd.read_excel('PCC6803 Promoter and reads 100bp.xlsx')
model=tf.keras.models.load_model('CNN model.h5')

2.	Define the 'changeseq', 'one_hot_encoding', and 'one_hot_encoding_mut' functions. The 'changeseq' randomly replaces nucleotides of the promoters with other nucleotides from the 'nucleotide' list. The 'one_hot_encoding' function returns the one-hot encoded data of promoter sequence and their logarithmically-scaled gene expression level. The 'one_hot_encoding_mut' mutates two regions of the promoter and returns the one-hot encoded data of the mutated promoter sequence by 'changeseq' function and their logarithmically-scaled promoter strengths.

In [ ]:
# 2. 

def changeseq(seq,start,ran):
    nucleotide = ['A','C','G','T']
    newseq = ''
    for i in range(ran):
        nucleotide = ['A','C','G','T']
        nucleotide.remove(seq[i+start])
        newseq += random.choice(nucleotide)
    return seq[:start]+newseq+seq[start+ran:]

def one_hot_encoding(df, seq_column, expression):
    bases = ['A','C','G','T']
    base_dict = dict(zip(bases,range(4)))
    n = len(df)
    total_width = df[seq_column].str.len().max()+20
    X = np.zeros((n,1,4,total_width))
    seqs = df[seq_column].values
    for i in range(n):
        seq = seqs[i]
        for b in range(len(seq)):
            X[i,0,base_dict[seq[b]], b+10+100-len(seq)] = 1    
    X = X.astype(theano.config.floatX)
    y = np.asarray(df[expression].values, dtype = theano.config.floatX)[:,np.newaxis]
    return X, y, total_width

def one_hot_encoding_mut(df, seq_column, expression, start1,start2,ran):
    bases = ['A','C','G','T']
    base_dict = dict(zip(bases,range(4)))
    n = len(df)
    total_width = df[seq_column].str.len().max()+20
    X = np.zeros((n,1,4,total_width))
    seqs = df[seq_column].values
    for i in range(n):
        seq = changeseq(seqs[i],start1,ran)
        seq = changeseq(seq,start2,ran)
        for b in range(len(seq)):
            X[i,0,base_dict[seq[b]], b+10+100-len(seq)] = 1    
    X = X.astype(theano.config.floatX)
    y = np.asarray(df[expression].values, dtype = theano.config.floatX)[:,np.newaxis]
    return X, y, total_width

3.	Critical subregion identification process. Based on the number of subregions (‘div’), the ‘subregion_size’ is calculated (div = [100/subregion size]). In the for-loop, two subregions are randomly selected and replaced with other sequences. This process is conducted for all the 3,712 native promoters. When the mutation dataset through 'one_hot_encoding_mut' function is generated, the Pearson correlation coefficient between the predicted strength of the native promoter and the mutated promoter dataset is calculated. This process is conducted for all possible subregion combinations (33C2 = 963 cases).

In [ ]:
# 3.

div = 33
subregion_size = int(100/div)
subregion_matrix = np.zeros((div-1,div-1))
for j in tqdm(range(div-1)):
    for k in range(div-1):
        arl = 0
        for i in range(10):
            X, y, total_width = one_hot_encoding(data,'Promoter','Reads')
            Xm, ym, total_widthm = one_hot_encoding_mut(data,'Promoter','Reads',subregion_size*j+2,subregion_size*k+2,rran)
            arl += pearsonr(model.predict(X)[:,0], model.predict(Xm)[:,0])[0]
        subregion_matrix[j][k] = arl/10
for i in range(div-1):
    subregion_matrix[i][i] = 1

4.	Draw the heatmap of the critical subregion identification analysis result. Because the result is symmetrical to the diagonal direction, the heatmap shows only the lower triangle part. 

In [ ]:
# 4.

plt.figure(figsize = (40,30))
plt.pcolor(resu,cmap = 'PiYG')
plt.title('3bp sequence mutation (3N)\n',fontsize = 120)
plt.xticks(np.arange(0.5,len(xy.index)),xy.index, rotation = 45,fontsize = 40)
plt.yticks(np.arange(0.5,len(xy.columns)),xy.columns,fontsize = 40)
plt.xlabel('\nMutated promoter sequence position',fontsize = 60)
plt.ylabel('Mutated promoter sequence position\n',fontsize = 60)
cbar = plt.colorbar()
cbar.set_label('\nPearson correlation coefficient',fontsize = 60)
tick_font_size = 40
cbar.ax.tick_params(labelsize=tick_font_size)